<a href="https://colab.research.google.com/github/Connor-Goodall/CrimeDetector/blob/main/CrimeProjectFinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import matplotlib
import pandas as pd
def loadCrimeData():
  crime = pd.read_csv("Crime_Data.csv")
  return crime
crime = loadCrimeData()

In [ ]:
crime

,RecordID,Offense,IncidentID,BlockNumber,StreetName,Agency,DateReported,HourReported,ReportingOfficer
0,1,Assault Simple,202200031540,900.0,GRADY AVE,CPD,2022/10/30 06:25:48+00,225,"Vlasis, Christopher"
1,2,Drunkeness DIP,202200031534,200.0,W WATER ST,CPD,2022/10/30 05:39:04+00,139,"Whitten, Tobyn"
2,3,Driving Under the Influence,202200031517,NaN,FONTAINE AVE / PIEDMONT AVE N,CPD,2022/10/30 03:29:39+00,2329,"Bentz, Benjamin"
3,4,Assault Simple,202200031508,700.0,"PROSPECT AVE, J",CPD,2022/10/30 02:36:00+00,2236,"Alfonso, Sean"
4,5,Larceny - All Other,202200031493,400.0,"W MAIN ST, A",CPD,2022/10/30 01:05:37+00,2105,"Darby, Michael"
...,...,...,...,...,...,...,...,...,...
24534,24535,Towed Vehicle,201700037259,1900.0,JEFFERSON PARK AVE,CPD,2017/11/01 13:59:00+00,959,"Sipe, Larry"
24535,24536,Suspicious Activity,201700037262,1000.0,CHERRY AVE,CPD,2017/11/01 13:30:00+00,930,"Culbreath, Jonathan"
24536,24537,Towed Vehicle,201700037251,400.0,14TH ST NW,CPD,2017/11/01 11:40:00+00,740,"Adldoost, Mohamad"
24537,24538,Assault Simple,201700037231,500.0,10TH ST NW,CPD,2017/11/01 07:30:00+00,330,"Blackwell, Kyle"


#Data Cleaning

In [ ]:
alphabet = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V"
, "W", "X", "Y", "Z"]
demo_locations = list(crime["StreetName"])
revised_street_names = []
#Goes through each street in the crime dataset and changes each street for better clarity for the website
#https://geocode.localfocus.nl/
for street in range(len(demo_locations)):
  word = ""
  currentWord = ""
  previousLetter = ""
  wordList = []
  for letter in range(len(demo_locations[street])):
    #If the street has a comma or a slash with a space, period, or any letter before it, do not continue with
    #the other street after these symbols 
    if (demo_locations[street][letter] == "," or (demo_locations[street][letter] == "/" and (previousLetter == " " or
      previousLetter == "." or previousLetter in alphabet))):
      break
    #If the street has a space with a @ before it, remove the street before the @
    elif(demo_locations[street][letter] == " " and previousLetter == "@"):
      word = ""
      wordList = []
    #If the street has a period in it, remove it from the street
    elif(demo_locations[street][letter] == "."):
      word += ""
    else:
      #If the street has a space, add the current word to the wordList
      if(demo_locations[street][letter] == " "):
        wordList.append(currentWord)
        currentWord = ""
      word += demo_locations[street][letter]
      currentWord += demo_locations[street][letter]
      #If the current word in the street is AVENUE, change it to AVE
      if(currentWord.strip() == "AVENUE"):
        word = word.replace(currentWord, " AVE")
        currentWord = " AVE"
      #If the current word in the street is STREET, change it to ST
      if(currentWord.strip() == "STREET"):
        word = word.replace(currentWord, " ST")
        currentWord = " ST"
      #If the current word in the street is a repeat, remove it from the street
      if(currentWord in wordList):
        word = word.replace(currentWord, "")
        break;
    previousLetter = demo_locations[street][letter]
  #Strip the word from all unnecessary spaces
  if(word.strip() in word):
    word = word.strip()
  revised_street_names.append(word)
crime["StreetName"] = revised_street_names

In [ ]:
all_streets = []
#For each street that has been revised
for street in revised_street_names:
  #If the street is not a repeat, add it to the all_streets list
  if(street not in all_streets):
    all_streets.append(street)

In [ ]:
#Used the website https://geocode.localfocus.nl/ to get the coordinates for the website
def loadCoordinatedData():
  coordinatedData = pd.read_csv("Coordinates.csv")
  return coordinatedData
coordinatedData = loadCoordinatedData()

In [ ]:
longitudeList = []
latitudeList = []
#For each coordinate in the coordinates dataset
for coordinate in range(len(coordinatedData)):
  #If the coordinate's name does not have Charlottesville in its name or has only Charlottesville, VA, USA 
  #as its name, change the longitude and latitude for this street to 0.00 
  if(not("Charlottesville" in coordinatedData["Name"][coordinate]) or coordinatedData["Name"][coordinate] == "Charlottesville, VA, USA"):
   longitudeList.append(0.00)
   latitudeList.append(0.00)
  else:
    longitudeList.append(coordinatedData["Longitude"][coordinate])
    latitudeList.append(coordinatedData["Latitude"][coordinate])


In [ ]:
realLongitudeList = []
realLatitudeList = []
#For each street in the crime dataset, give it its longitude and latitude
for street in crime["StreetName"]:
  index = all_streets.index(street)
  realLongitudeList.append(longitudeList[index])
  realLatitudeList.append(latitudeList[index])
#Add the longitude and latitude for each street to the crime dataset
crime["Longitude"] = realLongitudeList
crime["Latitude"] = realLatitudeList

In [ ]:
indexList =[]
#For each street in the crime dataset if the street's longitude and latitude is 0.000, remove it from the 
#crimes dataset 
for crimes in range(len(crime)):
  if(crime["Longitude"][crimes] == 0.000000 and crime["Latitude"][crimes] == 0.000000):
    indexList.append(crimes)
crime = crime.drop(indexList)


In [ ]:
#Dropped the BlockNumber feature since it is not relevant to the variable we are looking for and changed the
#RecordID feature to help with feature engineering in later parts
crime = crime.drop(["BlockNumber"], axis = 1)
recordCountList = []
recordCount = 0
for crimes in range(len(crime)):
  recordCountList.append(recordCount)
  recordCount += 1
crime["RecordID"] = recordCountList
crime.index = recordCountList


In [ ]:
violentCrimeList = []
nonviolentCrimeList = []
timeOfDayList = []
#Goes through each offense in the crime dataset and determines if it is a violent crime or not
for crimesDone in range(len(crime)):
  #If the crime is aggravated assault, rape, robbery, or homicide, then it is a violent crime and is not a nonviolent crime
  if(("Assault Aggravated" in crime["Offense"][crimesDone]) or ("Rape" in crime["Offense"][crimesDone]) or 
     ("Robbery" in crime["Offense"][crimesDone]) or ("Homicide" in crime["Offense"][crimesDone])):
        violentCrimeList.append("Y")
        nonviolentCrimeList.append("N")
  #If the crime is not any of the offenses above, then it is a nonviolent crime and is not a violent crime
  else:
    violentCrimeList.append("N")
    nonviolentCrimeList.append("Y")

  # Classify Crime as Morning, Afternoon or Evening
  hour = int(crime.iloc[crimesDone]["DateReported"].split(" ")[1].split(":")[0]) #[0,24)
  if hour >= 4 and hour <= 10:
    timeOfDayList.append("MORNING")
  elif hour >= 11 and hour <= 7:
    timeOfDayList.append("AFTERNOON")
  else:
    timeOfDayList.append("EVENING")

crime["ViolentCrimes"] = violentCrimeList
crime["NonviolentCrimes"] = nonviolentCrimeList
crime["TimeOfDay"] = timeOfDayList

In [ ]:
block_crimes = dict()
block_vio_crimes = dict()
block_nonvio_crimes = dict()
block_pops = dict()
blocks = []

for i in range(len(crime.index)):
  payload = {'lat': crime.iloc[i]["Latitude"], 'lon': crime.iloc[i]["Longitude"], 'censusYear': '2020', 'format': 'json'}
  url = "https://geo.fcc.gov/api/census/area?"
  response = requests.get(url, payload)

  payload2 = {'benchmark': 'Public_AR_Census2020', 'vintage': 'Census2020_Census2020', 'format': 'json',
             'x': crime.iloc[i]["Longitude"], 'y': crime.iloc[i]["Latitude"]}
  url2 = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?'
  response2 = requests.get(url2, payload2)
  try:
    block = int(response2.json()['result']['geographies']['Census Blocks'][0]['BLOCK'])
    blocks.append(block)
    
    block_pops[block] = int(response.json()['results'][0]['block_pop_2020'])
    
    if crime.iloc[i]["ViolentCrimes"] == "Y":
      if block in block_vio_crimes:
        block_vio_crimes[block] += 1
      else: block_vio_crimes[block] = 1
    else:
      if block in block_nonvio_crimes:
        block_nonvio_crimes[block] += 1
      else: block_nonvio_crimes[block] = 1

    if block in block_crimes:
        block_crimes[block] += 1
    else: block_crimes[block] = 1

    print(i)

  except:
    print(response.json())

Streaming output truncated to the last 5000 lines.
15645
15646
15647
15648
15649
15650
15651
15652
15653
15654
15655
15656
15657
15658
15659
15660
15661
15662
15663
15664
15665
15666
15667
15668
15669
15670
15671
15672
15673
15674
15675
15676
15677
15678
15679
15680
15681
15682
15683
15684
15685
15686
15687
15688
15689
15690
15691
15692
15693
15694
15695
15696
15697
15698
15699
15700
15701
15702
15703
15704
15705
15706
15707
15708
15709
15710
15711
15712
15713
15714
15715
15716
15717
15718
15719
15720
15721
15722
15723
15724
15725
15726
15727
15728
15729
15730
15731
15732
15733
15734
15735
15736
15737
15738
15739
15740
15741
15742
15743
15744
15745
15746
15747
15748
15749
15750
15751
15752
15753
15754
15755
15756
15757
15758
15759
15760
15761
15762
15763
15764
15765
15766
15767
15768
15769
15770
15771
15772
15773
15774
15775
15776
15777
15778
15779
15780
15781
15782
15783
15784
15785
15786
15787
15788
15789
15790
15791
15792
15793
15794
15795
15796
15797
15798
15799
15800
15801
15802
1

In [ ]:
crime["Blocks"] = blocks

NameError: ignored

In [ ]:
for each in block_crimes:
  if each not in block_vio_crimes:
    block_vio_crimes[each] = 0
  if each not in block_nonvio_crimes:
    block_nonvio_crimes[each] = 0

In [ ]:
no_pop = []
for each in block_pops:
  if block_pops[each] == 0:
    no_pop.append(each)

In [ ]:
# Replaces below code block
violentPercentDictionary = {}
nonviolentPercentDictionary = {}

for each in block_vio_crimes:
  if each not in no_pop:
    block_vio_crimes[each] = block_vio_crimes[each] / block_pops[each]
  else:
    block_vio_crimes[each] = block_vio_crimes[each] / 20  # Use default value of 20 population when that block has no population

for each in block_nonvio_crimes:
  if each not in no_pop:
    block_nonvio_crimes[each] = block_nonvio_crimes[each] / block_pops[each]
  else:
    block_vio_crimes[each] = block_vio_crimes[each] / 20  # Use default value of 20 population when that block has no population




In [ ]:
crime["ViolentCrimePercentPerStreet"] = ViolentPercentList 
crime["NonviolentCrimePercentPerStreet"] = NonviolentPercentList

#Get the percentage by multiplying by 100
crime["ViolentCrimePercentPerStreet"] = crime["ViolentCrimePercentPerStreet"] * 100 
crime["NonviolentCrimePercentPerStreet"] = crime["NonviolentCrimePercentPerStreet"] * 100

NameError: ignored

In [ ]:
ViolentPercentDictionary = {}
NonviolentPercentDictionary = {}
trainViolentCrimes = 0
testViolentCrimes = 0
trainNonviolentCrimes = 0
testNonviolentCrimes = 0
#Goes through each offense reported in the set and finds the total amount of violent and nonviolent crimes
# in each street and in the whole city of Charlottesville 
for crimesDone in range(len(crime)):
    #If the crime is a violent crime
    if(crime["ViolentCrimes"][crimesDone] == "Y"):
      #If the street that the crime took place at is not in the dictionary, add it to the dictionary
      if(ViolentPercentDictionary.get(crime["Blocks"][crimesDone]) == None):
        ViolentPercentDictionary[crime["Blocks"][crimesDone]] = 1
      else:
        #Add to the total violent crime in this certain street
        ViolentPercentDictionary[crime["Blocks"][crimesDone]] = ViolentPercentDictionary.get(crime["Blocks"][crimesDone]) + 1
    #If the crime is a nonviolent crime
    else:
      #If the street that the crime took place at is not in the dictionary, add it to the dictionary
      if(NonviolentPercentDictionary.get(crime["Blocks"][crimesDone]) == None):
        NonviolentPercentDictionary[crime["Blocks"][crimesDone]] = 1
      else:
        #Add to the total nonviolent crime in this street
        NonviolentPercentDictionary[crime["Blocks"][crimesDone]] = NonviolentPercentDictionary.get(crime["Blocks"][crimesDone]) + 1


#Go through each street in the set and divide the total violent crime in that street by the estimated population 
#of that street
for ViolentPercent in ViolentPercentDictionary:
  if block_pops[ViolentPercent] != 0:
    ViolentPercentDictionary[ViolentPercent] = ViolentPercentDictionary[ViolentPercent] / block_pops[ViolentPercent]
  else:
    ViolentPercentDictionary[ViolentPercent] = ViolentPercentDictionary[ViolentPercent] / 20

#Go through each street in the set and divide the total nonviolent crime in that street by the estimated population
# of that street
for NonviolentPercent in NonviolentPercentDictionary:
  if block_pops[NonviolentPercent] != 0:
    NonviolentPercentDictionary[NonviolentPercent] = NonviolentPercentDictionary[NonviolentPercent] / block_pops[NonviolentPercent]
  else:
    NonviolentPercentDictionary[NonviolentPercent] = NonviolentPercentDictionary[NonviolentPercent] / 20

ViolentPercentList = []
NonviolentPercentList = []

#Goes through each offense in the set and places the proportions for the amount of violent and nonviolent crimes
#for each street into their respective lists
for m in crime["RecordID"]:
  #If the crime is a violent crime
  if(crime["ViolentCrimes"][m] == "Y"):
    #Add the proportion for the amount of violent crimes for the street where the crime took place into the violent crime
    #list
    ViolentPercentList.append(ViolentPercentDictionary[crime["Blocks"][m]])
    #If the street does not have an associated proportion for the amount of nonviolent crimes, put 0
    if(NonviolentPercentDictionary.get(crime["Blocks"][m]) == None):
      NonviolentPercentList.append(0)
    #If the street does have an associated proportion for the amount of nonviolent crimes, add it to the nonviolent crime
    #list
    else:
     NonviolentPercentList.append(NonviolentPercentDictionary[crime["Blocks"][m]])
  #If the crime is a nonviolent crime
  else:
    #Add the proportion for the amount of nonviolent crimes for the street where the crime took place into the nonviolent 
    #crime list
    NonviolentPercentList.append(NonviolentPercentDictionary[crime["Blocks"][m]])
    #If the street does not have an associated proportion for the amount of violent crimes, put 0
    if(ViolentPercentDictionary.get(crime["Blocks"][m]) == None):
      ViolentPercentList.append(0)
    #If the street does have an associated proportion for the amount of violent crimes, add it to the violent crime
    #list
    else:
      ViolentPercentList.append(ViolentPercentDictionary[crime["Blocks"][m]])



crime["ViolentCrimePercentPerStreet"] = ViolentPercentList 
crime["NonviolentCrimePercentPerStreet"] = NonviolentPercentList

#Get the percentage by multiplying by 100
crime["ViolentCrimePercentPerStreet"] = crime["ViolentCrimePercentPerStreet"] * 100 
crime["NonviolentCrimePercentPerStreet"] = crime["NonviolentCrimePercentPerStreet"] * 100

KeyError: ignored

In [ ]:
ViolentPercentDictionary = {}
NonviolentPercentDictionary = {}
trainViolentCrimes = 0
testViolentCrimes = 0
trainNonviolentCrimes = 0
testNonviolentCrimes = 0
#Goes through each offense reported in the set and finds the total amount of violent and nonviolent crimes
# in each street and in the whole city of Charlottesville 
for crimesDone in range(len(crime)):
    #If the crime is a violent crime
    if(crime["ViolentCrimes"][crimesDone] == "Y"):
      #If the street that the crime took place at is not in the dictionary, add it to the dictionary
      if(ViolentPercentDictionary.get(crime["StreetName"][crimesDone]) == None):
        ViolentPercentDictionary[crime["StreetName"][crimesDone]] = 1
      else:
        #Add to the total violent crime in this certain street
        ViolentPercentDictionary[crime["StreetName"][crimesDone]] = ViolentPercentDictionary.get(crime["StreetName"][crimesDone]) + 1
    #If the crime is a nonviolent crime
    else:
      #If the street that the crime took place at is not in the dictionary, add it to the dictionary
      if(NonviolentPercentDictionary.get(crime["StreetName"][crimesDone]) == None):
        NonviolentPercentDictionary[crime["StreetName"][crimesDone]] = 1
      else:
        #Add to the total nonviolent crime in this street
        NonviolentPercentDictionary[crime["StreetName"][crimesDone]] = NonviolentPercentDictionary.get(crime["StreetName"][crimesDone]) + 1


#Go through each street in the set and divide the total violent crime in that street by the estimated population 
#of that street
for ViolentPercent in ViolentPercentDictionary:
  ViolentPercentDictionary[ViolentPercent] = ViolentPercentDictionary[ViolentPercent] / 200

#Go through each street in the set and divide the total nonviolent crime in that street by the estimated population
# of that street
for NonviolentPercent in NonviolentPercentDictionary:
  NonviolentPercentDictionary[NonviolentPercent] = NonviolentPercentDictionary[NonviolentPercent] / 200

ViolentPercentList = []
NonviolentPercentList = []

#Goes through each offense in the set and places the proportions for the amount of violent and nonviolent crimes
#for each street into their respective lists
for m in crime["RecordID"]:
  #If the crime is a violent crime
  if(crime["ViolentCrimes"][m] == "Y"):
    #Add the proportion for the amount of violent crimes for the street where the crime took place into the violent crime
    #list
    ViolentPercentList.append(ViolentPercentDictionary[crime["StreetName"][m]])
    #If the street does not have an associated proportion for the amount of nonviolent crimes, put 0
    if(NonviolentPercentDictionary.get(crime["StreetName"][m]) == None):
      NonviolentPercentList.append(0)
    #If the street does have an associated proportion for the amount of nonviolent crimes, add it to the nonviolent crime
    #list
    else:
     NonviolentPercentList.append(NonviolentPercentDictionary[crime["StreetName"][m]])
  #If the crime is a nonviolent crime
  else:
    #Add the proportion for the amount of nonviolent crimes for the street where the crime took place into the nonviolent 
    #crime list
    NonviolentPercentList.append(NonviolentPercentDictionary[crime["StreetName"][m]])
    #If the street does not have an associated proportion for the amount of violent crimes, put 0
    if(ViolentPercentDictionary.get(crime["StreetName"][m]) == None):
      ViolentPercentList.append(0)
    #If the street does have an associated proportion for the amount of violent crimes, add it to the violent crime
    #list
    else:
      ViolentPercentList.append(ViolentPercentDictionary[crime["StreetName"][m]])



crime["ViolentCrimePercentPerStreet"] = ViolentPercentList 
crime["NonviolentCrimePercentPerStreet"] = NonviolentPercentList

#Get the percentage by multiplying by 100
crime["ViolentCrimePercentPerStreet"] = crime["ViolentCrimePercentPerStreet"] * 100 
crime["NonviolentCrimePercentPerStreet"] = crime["NonviolentCrimePercentPerStreet"] * 100


In [ ]:
import copy

def get_crime_ds():
  return copy.deepcopy(crime)

In [ ]:
from sklearn.model_selection import train_test_split
#Split the crime dataset into a training and test sets
split_set, crime_test_set = train_test_split(crime, test_size = 0.1, random_state=14)
crime_train_set, crime_validation_set = train_test_split(split_set, test_size=0.1, random_state=14)

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.compose import ColumnTransformer
crime_training_set = crime_train_set.drop(["ViolentCrimePercentPerStreet", "NonviolentCrimePercentPerStreet", "RecordID"], axis = 1)
violent_crime_train_percent = crime_train_set["ViolentCrimePercentPerStreet"].copy()
nonviolent_crime_train_percent = crime_train_set["NonviolentCrimePercentPerStreet"].copy()
crime_testing_set = crime_test_set.drop(["ViolentCrimePercentPerStreet", "NonviolentCrimePercentPerStreet", "RecordID"], axis = 1)
violent_crime_test_percent = crime_test_set["ViolentCrimePercentPerStreet"].copy()
nonviolent_crime_test_percent = crime_test_set["NonviolentCrimePercentPerStreet"].copy()
crime_num = crime_training_set.select_dtypes(include=[np.number])
numerical_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")), ('std_scaler', StandardScaler())])
crime_cat = crime_training_set.select_dtypes(include=[object])
num_attribs = list(crime_num)
cat_attribs = list(crime_cat)
full_pipeline = ColumnTransformer([('num', numerical_pipeline, num_attribs), ("cat", OneHotEncoder(handle_unknown = 'ignore'), cat_attribs)], )
crime_training_set_prepared = full_pipeline.fit_transform(crime_training_set)
crime_testing_set_prepared = full_pipeline.transform(crime_testing_set)

# Data Visualization

#### SETUP

testing how to use the mapping packages

In [ ]:
import folium

def get_cvile_map():
  return folium.Map(location=[38.033554, -78.507980], zoom_start=15, max_zoom=25, width="70%", height="60%") #UVA rotunda building

map = get_cvile_map()

In [ ]:
crime_ds = get_crime_ds()

crime_ds.head()

NameError: ignored

In [ ]:
a_crime = crime_ds.iloc()[0]

In [ ]:
print(a_crime["DateReported"].split(" ")[1].split(":")[0])

In [ ]:
popup_str = a_crime["Offense"] + " - " + a_crime["DateReported"]
folium.Marker([a_crime["Latitude"], a_crime["Longitude"]], popup=popup_str).add_to(map)

map

## Visulation of Violent Crimes in Charlottesville 

In [ ]:
violent_crimes=get_crime_ds().query("ViolentCrimes == 'Y'")
violent_crimes.head()

,RecordID,Offense,IncidentID,StreetName,Agency,DateReported,HourReported,ReportingOfficer,Longitude,Latitude,ViolentCrimes,NonviolentCrimes,TimeOfDay,Blocks,ViolentCrimePercentPerStreet,NonviolentCrimePercentPerStreet
51,51,Assault Aggravated,202200030944,RIDGE ST,CPD,2022/10/25 22:46:00+00,1846,"Venereo, Arturo",-78.485196,38.030426,Y,N,EVENING,1037,11.111111,265.972222
54,54,Assault Aggravated,202200030914,NASSAU ST,CPD,2022/10/25 18:06:55+00,1406,"Pendleton, Eric",-78.467818,38.018075,Y,N,EVENING,3010,10.000000,128.000000
87,87,Assault Aggravated,202200030660,PRESTON AVE,CPD,2022/10/23 17:35:08+00,1335,"Hopiane, Sayd",-78.489639,38.037756,Y,N,EVENING,1000,6.963788,236.490251
93,93,Assault Aggravated,202200030643,W MAIN ST,CPD,2022/10/23 04:55:05+00,55,"Whitten, Tobyn",-78.485833,38.030494,Y,N,MORNING,1038,145.000000,5770.000000
118,118,Assault Aggravated,202200030382,LANKFORD AVE,CPD,2022/10/21 02:12:21+00,2212,"Blackwell, Ryan",-78.488728,38.022367,Y,N,EVENING,2003,4.112406,107.882111


In [ ]:
violent_map = get_cvile_map()

for index, a_crime in violent_crimes.iterrows():
  if not a_crime["Latitude"] or not a_crime["Longitude"]:
    continue

  hour = int(a_crime["DateReported"].split(" ")[1].split(":")[0]) #[0,24)

  meta_data = {}

  if a_crime["TimeOfDay"] == "MORNING":
    meta_data["time"] = "morning"
    meta_data["color"] = "lightblue"
    meta_data["icon"] = "fa-coffee"
  elif a_crime["TimeOfDay"] == "AFTERNOON":
    meta_data["time"] = "afternoon"
    meta_data["color"] = "green"
    meta_data["icon"] = "fa-sun-o"
  else:
    meta_data["time"] = "night"
    meta_data["color"] = "darkpurple"
    meta_data["icon"] = "fa-moon-o"

  popup_str = a_crime["Offense"] + " - " + a_crime["DateReported"]
  folium.Marker([a_crime["Latitude"], a_crime["Longitude"]], popup=popup_str, icon=folium.Icon(color=meta_data["color"], prefix='fa', icon=meta_data["icon"])).add_to(violent_map)

violent_map

## Visualization of Non-Violent Crimes in Charelottesville 

In [ ]:
non_violent_crimes=get_crime_ds().query("ViolentCrimes == 'N'")
non_violent_crimes

NameError: ignored

In [ ]:
NUM_CRIMES = 5000 #Mapping software only accepts so many points
non_violent_map = get_cvile_map()

counter = 0
for index, a_crime in non_violent_crimes.iterrows():
  if not a_crime["Latitude"] or not a_crime["Longitude"]:
    continue

  hour = int(a_crime["DateReported"].split(" ")[1].split(":")[0]) #[0,24)

  if counter == NUM_CRIMES:
    break

  meta_data = {}

  if a_crime["TimeOfDay"] == "MORNING":
    meta_data["time"] = "morning"
    meta_data["color"] = "lightblue"
    meta_data["icon"] = "fa-coffee"
  elif a_crime["TimeOfDay"] == "AFTERNOON":
    meta_data["time"] = "afternoon"
    meta_data["color"] = "green"
    meta_data["icon"] = "fa-sun-o"
  else:
    meta_data["time"] = "night"
    meta_data["color"] = "darkpurple"
    meta_data["icon"] = "fa-moon-o"

  popup_str = a_crime["Offense"] + " - " + a_crime["DateReported"]
  folium.Marker([a_crime["Latitude"], a_crime["Longitude"]], popup=popup_str, icon=folium.Icon(color=meta_data["color"], prefix='fa', icon=meta_data["icon"])).add_to(non_violent_map)

  counter += 1

non_violent_map

NameError: ignored

Note that no crimes are reported in the afternnon hours between 11am-8pm

# Model Selection

***Violent Crimes***

*Linear Regression*

In [ ]:
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error
lin_reg = LinearRegression()
lin_reg.fit(crime_training_set_prepared, violent_crime_train_percent)
linear_training_predictions = lin_reg.predict(crime_training_set_prepared)
lin_mse = mean_squared_error(violent_crime_train_percent, linear_training_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

NameError: ignored

In [ ]:
from sklearn.model_selection import cross_val_score
linear_scores = cross_val_score(lin_reg, crime_training_set_prepared, violent_crime_train_percent, scoring ="neg_mean_squared_error", cv=5)
pd.Series(np.sqrt(-linear_scores)).describe()

count    5.000000
mean     2.083523
std      0.265384
min      1.746357
25%      1.864555
50%      2.195618
75%      2.238708
max      2.372380
dtype: float64

*Decision Tree Regression*

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(random_state=14)
tree_reg.fit(crime_training_set_prepared, violent_crime_train_percent)
tree_training_predictions = tree_reg.predict(crime_training_set_prepared)
tree_mse = mean_squared_error(violent_crime_train_percent, tree_training_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

3.139803648854946e-14

In [ ]:
tree_scores = cross_val_score(tree_reg, crime_training_set_prepared, violent_crime_train_percent, scoring ="neg_mean_squared_error", cv=5)
pd.Series(np.sqrt(-tree_scores)).describe()

count    5.000000
mean     0.851842
std      0.312277
min      0.536942
25%      0.707525
50%      0.796273
75%      0.849922
max      1.368549
dtype: float64

*Random Forest Regression*

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
param_distribs = {
        'n_estimators': randint(low=1, high=200),
    }
forest_reg = RandomForestRegressor(random_state=14)
forest_violent_rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=14)
forest_violent_rnd_search.fit(crime_training_set_prepared, violent_crime_train_percent)
forest_violent_cvres = forest_violent_rnd_search.cv_results_
for mean_score, params in zip(forest_violent_cvres["mean_test_score"], forest_violent_cvres["params"]):
    print(np.sqrt(-mean_score), params)

1.7189178762566522 {'n_estimators': 108}
1.7332922206591448 {'n_estimators': 89}
1.7281375042172444 {'n_estimators': 13}
1.6806584972155794 {'n_estimators': 151}
1.7315189001901257 {'n_estimators': 72}
1.7188484729823565 {'n_estimators': 103}
1.6934424403377732 {'n_estimators': 157}
1.7003740320901297 {'n_estimators': 139}
1.7509793045470825 {'n_estimators': 62}
1.7145617630505203 {'n_estimators': 110}


In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(n_estimators=157, random_state=14)
forest_reg.fit(crime_training_set_prepared, violent_crime_train_percent)
forest_crime_predictions = forest_reg.predict(crime_training_set_prepared)
forest_mse = mean_squared_error(violent_crime_train_percent, forest_crime_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

0.4178982815498981

In [ ]:
forest_scores = cross_val_score(forest_reg, crime_training_set_prepared, violent_crime_train_percent, scoring ="neg_mean_squared_error", cv=5)
pd.Series(np.sqrt(-forest_scores)).describe()

count    5.000000
mean     1.685003
std      0.188786
min      1.400969
25%      1.596882
50%      1.766155
75%      1.779550
max      1.881458
dtype: float64

*SGD Regression*

In [ ]:
from sklearn.linear_model import SGDRegressor
from scipy.stats import uniform
param_distribs = {
        'alpha': uniform(loc=0.001),
        'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
        'eta0': uniform(loc=0.001)
    }
sgd_reg = SGDRegressor(random_state=14)
sgd_violent_rnd_search = RandomizedSearchCV(sgd_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=14)
sgd_violent_rnd_search.fit(crime_training_set_prepared, violent_crime_train_percent)
sgd_violent_cvres = sgd_violent_rnd_search.cv_results_
for mean_score, params in zip(sgd_violent_cvres["mean_test_score"], sgd_violent_cvres["params"]):
    print(np.sqrt(-mean_score), params)

45.64956508077038 {'alpha': 0.5149433437726902, 'eta0': 0.7741650520792968, 'learning_rate': 'adaptive'}
44.77639633291528 {'alpha': 0.36728239607845947, 'eta0': 0.15000510988932791, 'learning_rate': 'invscaling'}
49.66210167646063 {'alpha': 0.9586037393658909, 'eta0': 0.5141167122678871, 'learning_rate': 'invscaling'}
1387997352481.7234 {'alpha': 0.36511977216005276, 'eta0': 0.5400126647148649, 'learning_rate': 'constant'}
48.769509809999704 {'alpha': 0.80748135792427, 'eta0': 0.34325462561468434, 'learning_rate': 'invscaling'}
48.0326193492594 {'alpha': 0.9673270712336435, 'eta0': 0.15242228210831044, 'learning_rate': 'optimal'}
41.65197253664101 {'alpha': 0.21102426217873815, 'eta0': 0.9335575928787119, 'learning_rate': 'adaptive'}
48.11587329214026 {'alpha': 0.9923350548775414, 'eta0': 0.227584753119491, 'learning_rate': 'optimal'}
47.63061508787512 {'alpha': 0.8714994552717501, 'eta0': 0.11111117825660166, 'learning_rate': 'invscaling'}
47.00556495129359 {'alpha': 0.80047421308536

In [ ]:
sgd_reg = SGDRegressor(random_state=14, alpha=0.21102426217873815, eta0 = 0.9335575928787119, learning_rate = "adaptive")
sgd_reg.fit(crime_training_set_prepared, violent_crime_train_percent)
sgd_crime_predictions = sgd_reg.predict(crime_training_set_prepared)
sgd_mse = mean_squared_error(violent_crime_train_percent, sgd_crime_predictions)
sgd_rmse = np.sqrt(sgd_mse)
sgd_rmse

41.59628529550964

In [ ]:
sgd_scores = cross_val_score(sgd_reg, crime_training_set_prepared, violent_crime_train_percent, scoring ="neg_mean_squared_error", cv=5)
pd.Series(np.sqrt(-sgd_scores)).describe()

count     5.000000
mean     41.648993
std       0.556991
min      40.838185
25%      41.440758
50%      41.655656
75%      42.037529
max      42.272838
dtype: float64

*SVM Regression*

In [ ]:
from sklearn.svm import SVR
param_distribs = {
        'kernel': ["linear", 'poly', 'rbf', 'sigmoid'],
        'coef0': randint(low = 1, high = 1000),
        'C': randint(low = 1, high = 1000)
    }
SVR_reg = SVR()
SVR_violent_rnd_search = RandomizedSearchCV(SVR_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=14)
SVR_violent_rnd_search.fit(crime_training_set_prepared, violent_crime_train_percent)
SVR_violent_cvres = SVR_violent_rnd_search.cv_results_
for mean_score, params in zip(SVR_violent_cvres["mean_test_score"], SVR_violent_cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
SVR_reg = SVR(kernel = 'linear', C = 620, coef0 = 345)
SVR_reg.fit(crime_training_set_prepared, violent_crime_train_percent)
SVR_crime_predictions = SVR_reg.predict(crime_training_set_prepared)
SVR_mse = mean_squared_error(violent_crime_train_percent, SVR_crime_predictions)
SVR_rmse = np.sqrt(SVR_mse)
SVR_rmse

In [ ]:
SVR_scores = cross_val_score(SVR_reg, crime_training_set_prepared, violent_crime_train_percent, scoring ="neg_mean_squared_error", cv=5)
pd.Series(np.sqrt(-SVR_scores)).describe()

*Lasso Regression*

In [ ]:
from sklearn import linear_model
param_distribs = {
        'alpha': uniform(loc=0.001),
    }
lasso_reg = linear_model.Lasso(random_state = 14)
lasso_violent_rnd_search = RandomizedSearchCV(lasso_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=14)
lasso_violent_rnd_search.fit(crime_training_set_prepared, violent_crime_train_percent)
lasso_violent_cvres = lasso_violent_rnd_search.cv_results_
for mean_score, params in zip(lasso_violent_cvres["mean_test_score"], lasso_violent_cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
lasso_reg = linear_model.Lasso(random_state = 14, alpha = 0.00904694852981791)
lasso_reg.fit(crime_training_set_prepared, violent_crime_train_percent)
lasso_crime_predictions = lasso_reg.predict(crime_training_set_prepared)
lasso_mse = mean_squared_error(violent_crime_train_percent, lasso_crime_predictions)
lasso_rmse = np.sqrt(lasso_mse)
lasso_rmse

In [ ]:
lasso_scores = cross_val_score(lasso_reg, crime_training_set_prepared, violent_crime_train_percent, scoring ="neg_mean_squared_error", cv=5)
pd.Series(np.sqrt(-lasso_scores)).describe()

*Ridge Regression*

In [ ]:
from sklearn.linear_model import Ridge
param_distribs = {
        'alpha': uniform(loc=0.001),
    }
ridge_reg = Ridge(random_state = 14)
ridge_violent_rnd_search = RandomizedSearchCV(ridge_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=14)
ridge_violent_rnd_search.fit(crime_training_set_prepared, violent_crime_train_percent)
ridge_violent_cvres = ridge_violent_rnd_search.cv_results_
for mean_score, params in zip(ridge_violent_cvres["mean_test_score"], ridge_violent_cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
ridge_reg = Ridge(random_state = 14, alpha = 0.00904694852981791)
ridge_reg.fit(crime_training_set_prepared, violent_crime_train_percent)
ridge_crime_predictions = ridge_reg.predict(crime_training_set_prepared)
ridge_mse = mean_squared_error(violent_crime_train_percent, ridge_crime_predictions)
ridge_rmse = np.sqrt(ridge_mse)
ridge_rmse

In [ ]:
ridge_scores = cross_val_score(ridge_reg, crime_training_set_prepared, violent_crime_train_percent, scoring ="neg_mean_squared_error", cv=5)
pd.Series(np.sqrt(-ridge_scores)).describe()

*Gradient Boosting Regression*

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'learning_rate': uniform(loc = 0.01)
    }
boosting_reg = GradientBoostingRegressor(random_state=14)
boosting_violent_rnd_search = RandomizedSearchCV(boosting_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=14)
boosting_violent_rnd_search.fit(crime_training_set_prepared, violent_crime_train_percent)
boosting_violent_cvres = boosting_violent_rnd_search.cv_results_
for mean_score, params in zip(boosting_violent_cvres["mean_test_score"], boosting_violent_cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
boosting_reg = GradientBoostingRegressor(n_estimators = 174, learning_rate = 0.7624189220859464, random_state = 14)
boosting_reg.fit(crime_training_set_prepared, violent_crime_train_percent)
boosting_crime_predictions = boosting_reg.predict(crime_training_set_prepared)
boosting_mse = mean_squared_error(violent_crime_train_percent, boosting_crime_predictions)
boosting_rmse = np.sqrt(boosting_mse)
boosting_rmse

In [ ]:
boosting_scores = cross_val_score(boosting_reg, crime_training_set_prepared, violent_crime_train_percent, scoring ="neg_mean_squared_error", cv=5)
pd.Series(np.sqrt(-boosting_scores)).describe()

*Best Model Based on Performance*

The best model out of the eight that were chosen for performance would be the random forest regression model since it has the smallest mean performance (RMSE) value out of all of the eight regression models used.

***Nonviolent Crimes***

*Linear Regression*

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(crime_training_set_prepared, nonviolent_crime_train_percent)
linear_training_predictions = lin_reg.predict(crime_training_set_prepared)
lin_mse = mean_squared_error(nonviolent_crime_train_percent, linear_training_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
linear_scores = cross_val_score(lin_reg, crime_training_set_prepared, nonviolent_crime_train_percent, scoring ="neg_mean_squared_error", cv=5)
pd.Series(np.sqrt(-linear_scores)).describe()

*Decision Tree Regression*

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(random_state=14)
tree_reg.fit(crime_training_set_prepared, nonviolent_crime_train_percent)
tree_training_predictions = tree_reg.predict(crime_training_set_prepared)
tree_mse = mean_squared_error(nonviolent_crime_train_percent, tree_training_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

In [ ]:
tree_scores = cross_val_score(tree_reg, crime_training_set_prepared, nonviolent_crime_train_percent, scoring ="neg_mean_squared_error", cv=5)
pd.Series(np.sqrt(-tree_scores)).describe()

*Random Forest Regression*

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
param_distribs = {
        'n_estimators': randint(low=1, high=200),
    }
forest_reg = RandomForestRegressor(random_state=14)
forest_nonviolent_rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=14)
forest_nonviolent_rnd_search.fit(crime_training_set_prepared, nonviolent_crime_train_percent)
forest_nonviolent_cvres = forest_nonviolent_rnd_search.cv_results_
for mean_score, params in zip(forest_nonviolent_cvres["mean_test_score"], forest_nonviolent_cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
forest_reg = RandomForestRegressor(n_estimators=108, random_state=14)
forest_reg.fit(crime_training_set_prepared, nonviolent_crime_train_percent)
forest_crime_predictions = forest_reg.predict(crime_training_set_prepared)
forest_mse = mean_squared_error(nonviolent_crime_train_percent, forest_crime_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [ ]:
forest_scores = cross_val_score(forest_reg, crime_training_set_prepared, nonviolent_crime_train_percent, scoring ="neg_mean_squared_error", cv=5)
pd.Series(np.sqrt(-forest_scores)).describe()

*SGD Regression*

In [ ]:
param_distribs = {
        'alpha': uniform(loc=0.001),
        'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
        'eta0': uniform(loc=0.001)
    }
sgd_reg = SGDRegressor(random_state=14)
sgd_nonviolent_rnd_search = RandomizedSearchCV(sgd_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=14)
sgd_nonviolent_rnd_search.fit(crime_training_set_prepared, nonviolent_crime_train_percent)
sgd_nonviolent_cvres = sgd_nonviolent_rnd_search.cv_results_
for mean_score, params in zip(sgd_nonviolent_cvres["mean_test_score"], sgd_nonviolent_cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
sgd_reg = SGDRegressor(random_state=14, alpha = 0.2110242621787381, eta0 = 0.9335575928787119, learning_rate = 'adaptive')
sgd_reg.fit(crime_training_set_prepared, nonviolent_crime_train_percent)
sgd_crime_predictions = sgd_reg.predict(crime_training_set_prepared)
sgd_mse = mean_squared_error(nonviolent_crime_train_percent, sgd_crime_predictions)
sgd_rmse = np.sqrt(sgd_mse)
sgd_rmse

In [ ]:
sgd_scores = cross_val_score(sgd_reg, crime_training_set_prepared, nonviolent_crime_train_percent, scoring ="neg_mean_squared_error", cv=5)
pd.Series(np.sqrt(-sgd_scores)).describe()

*SVM Regression*

In [ ]:
param_distribs = {
        'kernel': ["linear", 'poly', 'rbf', 'sigmoid'],
        'coef0': randint(low = 1, high = 1000),
        'C': randint(low = 1, high = 1000)
    }
SVR_reg = SVR()
SVR_nonviolent_rnd_search = RandomizedSearchCV(SVR_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=14)
SVR_nonviolent_rnd_search.fit(crime_training_set_prepared, nonviolent_crime_train_percent)
SVR_nonviolent_cvres = SVR_nonviolent_rnd_search.cv_results_
for mean_score, params in zip(SVR_nonviolent_cvres["mean_test_score"], SVR_nonviolent_cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
SVR_reg = SVR(kernel = 'linear', C = 359, coef0 = 747)
SVR_reg.fit(crime_training_set_prepared, nonviolent_crime_train_percent)
SVR_crime_predictions = SVR_reg.predict(crime_training_set_prepared)
SVR_mse = mean_squared_error(nonviolent_crime_train_percent, SVR_crime_predictions)
SVR_rmse = np.sqrt(SVR_mse)
SVR_rmse

In [ ]:
SVR_scores = cross_val_score(SVR_reg, crime_training_set_prepared, nonviolent_crime_train_percent, scoring ="neg_mean_squared_error", cv=5)
pd.Series(np.sqrt(-SVR_scores)).describe()

*Lasso Regression*

In [ ]:
param_distribs = {
        'alpha': uniform(loc=0.001),
    }
lasso_reg = linear_model.Lasso(random_state = 14)
lasso_nonviolent_rnd_search = RandomizedSearchCV(lasso_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=14)
lasso_nonviolent_rnd_search.fit(crime_training_set_prepared, nonviolent_crime_train_percent)
lasso_nonviolent_cvres = lasso_nonviolent_rnd_search.cv_results_
for mean_score, params in zip(lasso_nonviolent_cvres["mean_test_score"], lasso_nonviolent_cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
lasso_reg = linear_model.Lasso(random_state = 14, alpha = 0.00904694852981791)
lasso_reg.fit(crime_training_set_prepared, nonviolent_crime_train_percent)
lasso_crime_predictions = lasso_reg.predict(crime_training_set_prepared)
lasso_mse = mean_squared_error(nonviolent_crime_train_percent, lasso_crime_predictions)
lasso_rmse = np.sqrt(lasso_mse)
lasso_rmse

In [ ]:
lasso_scores = cross_val_score(lasso_reg, crime_training_set_prepared, nonviolent_crime_train_percent, scoring ="neg_mean_squared_error", cv=5)
pd.Series(np.sqrt(-lasso_scores)).describe()

*Ridge Regression*

In [ ]:
param_distribs = {
        'alpha': uniform(loc=0.001),
    }
ridge_reg = Ridge(random_state = 14)
ridge_nonviolent_rnd_search = RandomizedSearchCV(ridge_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=14)
ridge_nonviolent_rnd_search.fit(crime_training_set_prepared, nonviolent_crime_train_percent)
ridge_nonviolent_cvres = ridge_nonviolent_rnd_search.cv_results_
for mean_score, params in zip(ridge_nonviolent_cvres["mean_test_score"], ridge_nonviolent_cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
ridge_reg = Ridge(random_state = 14, alpha = 0.00904694852981791)
ridge_reg.fit(crime_training_set_prepared, nonviolent_crime_train_percent)
ridge_crime_predictions = ridge_reg.predict(crime_training_set_prepared)
ridge_mse = mean_squared_error(nonviolent_crime_train_percent, ridge_crime_predictions)
ridge_rmse = np.sqrt(ridge_mse)
ridge_rmse

In [ ]:
ridge_scores = cross_val_score(ridge_reg, crime_training_set_prepared, nonviolent_crime_train_percent, scoring ="neg_mean_squared_error", cv=5)
pd.Series(np.sqrt(-ridge_scores)).describe()

*Gradient Boosting Regression*

In [ ]:
param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'learning_rate': uniform(loc = 0.01)
    }
boosting_reg = GradientBoostingRegressor(random_state=14)
boosting_nonviolent_rnd_search = RandomizedSearchCV(boosting_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=14)
boosting_nonviolent_rnd_search.fit(crime_training_set_prepared, nonviolent_crime_train_percent)
boosting_nonviolent_cvres = boosting_nonviolent_rnd_search.cv_results_
for mean_score, params in zip(boosting_nonviolent_cvres["mean_test_score"], boosting_nonviolent_cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
boosting_reg = GradientBoostingRegressor(n_estimators = 110, random_state = 14, learning_rate = 0.967603739365891)
boosting_reg.fit(crime_training_set_prepared, nonviolent_crime_train_percent)
boosting_crime_predictions = boosting_reg.predict(crime_training_set_prepared)
boosting_mse = mean_squared_error(nonviolent_crime_train_percent, boosting_crime_predictions)
boosting_rmse = np.sqrt(boosting_mse)
boosting_rmse

In [ ]:
boosting_scores = cross_val_score(boosting_reg, crime_training_set_prepared, nonviolent_crime_train_percent, scoring ="neg_mean_squared_error", cv=5)
pd.Series(np.sqrt(-boosting_scores)).describe()

*Best Model Based on Performance*

The best model out of the eight that were chosen for performance would be the decision tree regression model since it has the smallest mean performance (RMSE) value out of all of the eight regression models used.

***Final Model***

*Violent Crimes*

In [ ]:
from sklearn.ensemble import RandomForestRegressor
final_violent_model = RandomForestRegressor(n_estimators=72, random_state=14)
final_violent_model.fit(crime_training_set_prepared, violent_crime_train_percent)
final_violent_predictions = final_violent_model.predict(crime_testing_set_prepared)
final_violent_mse = mean_squared_error(violent_crime_test_percent, final_violent_predictions)
final_violent_rmse = np.sqrt(final_violent_mse)
final_violent_rmse

0.016652368093050544

In [ ]:
sorted_violent_predictions = np.argsort(final_violent_predictions)
n = 1
print(sorted_violent_predictions[-n], sorted_violent_predictions[-n - 1], sorted_violent_predictions[-n - 2], sorted_violent_predictions[-n - 3], 
      sorted_violent_predictions[-n] - 4, sorted_violent_predictions[-n] - 5, sorted_violent_predictions[-n] - 6, sorted_violent_predictions[-n] - 7,
      sorted_violent_predictions[-n] - 8)
print(final_violent_predictions[486], final_violent_predictions[572], final_violent_predictions[1792], final_violent_predictions[775], 
      final_violent_predictions[482],  final_violent_predictions[481], final_violent_predictions[480], final_violent_predictions[479],
      final_violent_predictions[478])
count = 0
for i in crime_test_set["StreetName"]:
  if(count == 486 or count == 572 or count == 1792 or count == 775 or count == 482 or count == 481 or count == 480 or count == 478):
    print(i)
  count += 1

486 576 1792 775 482 481 480 479 478
16.5 6.5 16.5 16.5 14.499999999999996 1.0 14.499999999999996 14.499999999999996 0.5
LOCUST AVE
W MAIN ST
CEDAR HILL RD
W MAIN ST
PROSPECT AVE
5TH ST SW
PROSPECT AVE
PROSPECT AVE


*Nonviolent Crimes*

In [ ]:
from sklearn.tree import DecisionTreeRegressor
final_nonviolent_model = DecisionTreeRegressor(random_state=14)
final_nonviolent_model.fit(crime_training_set_prepared, nonviolent_crime_train_percent)
final_nonviolent_predictions = final_nonviolent_model.predict(crime_testing_set_prepared)
final_nonviolent_mse = mean_squared_error(nonviolent_crime_test_percent, final_nonviolent_predictions)
final_nonviolent_rmse = np.sqrt(final_nonviolent_mse)
final_nonviolent_rmse

0.8483711783024775

In [ ]:
sorted_nonviolent_predictions = np.argsort(final_nonviolent_predictions)
n = 1
print(sorted_nonviolent_predictions[-n], sorted_nonviolent_predictions[-n - 1], sorted_nonviolent_predictions[-n - 2], sorted_nonviolent_predictions[-n - 3], 
      sorted_nonviolent_predictions[-n] - 4, sorted_nonviolent_predictions[-n] - 5, sorted_nonviolent_predictions[-n] - 6, sorted_nonviolent_predictions[-n] - 7,
      sorted_nonviolent_predictions[-n] - 8, sorted_nonviolent_predictions[-n] - 9, sorted_nonviolent_predictions[-n] - 10)
print(final_nonviolent_predictions[1468], final_nonviolent_predictions[1636], final_nonviolent_predictions[1350], final_nonviolent_predictions[906], 
      final_nonviolent_predictions[1464],  final_nonviolent_predictions[1463], final_nonviolent_predictions[1462], final_nonviolent_predictions[1461],
      final_nonviolent_predictions[1460], final_nonviolent_predictions[1459], final_nonviolent_predictions[1458])
count = 0
for i in crime_test_set["StreetName"]:
  if(count == 1468 or count == 1636 or count == 1350 or count == 906 or count == 1464 or count == 1463 or count == 1462 or count == 1461 or count == 1459 or 
     count == 1458):
    print(i)
    print(count)
  count += 1

1468 1636 1350 906 1464 1463 1462 1461 1460 1459 1458
577.0 577.0 577.0 577.0 157.0 262.0 577.0 262.0 577.0 6.0 203.5
W MAIN ST
906
W MAIN ST
1350
PRESTON AVE
1458
OAKHURST CIR
1459
E MAIN ST
1461
W MAIN ST
1462
E MAIN ST
1463
CARLTON AVE
1464
W MAIN ST
1468
W MAIN ST
1636
